In [ ]:
import pandas as pd
df= pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=45)

In [ ]:
x_train.affordibility[0]

1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
x_train_scaled = x_train.copy()
x_train_scaled['age'] = scalar.fit_transform(x_train_scaled[['age']])
x_train_scaled

,age,affordibility
7,0.954545,0
9,0.977273,1
22,0.500000,1
6,0.840909,0
20,0.068182,1
16,0.159091,0
17,0.909091,1
2,0.659091,1
12,0.204545,0
5,0.863636,1


In [ ]:
from tensorflow import keras
model = keras.Sequential(
    [keras.layers.Dense(1, input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')]
)

model.compile(
    optimizer='adam',loss='binary_crossentropy', metrics=['accuracy']
)
model.fit(x_train_scaled,y_train,epochs=2000)

In [ ]:
cofficient, intercept = model.get_weights()
cofficient, intercept

(array([[2.8387413],
        [0.5795002]], dtype=float32), array([-1.4592602], dtype=float32))

In [ ]:
import numpy as np
def sigmoid(weighted_sum):
  output= 1/(1+np.exp(-weighted_sum))
  return output

In [ ]:
sigmoid(8)

0.9996646498695336

In [ ]:
def predicted(x1,x2):
  w1=cofficient[0]
  w2=cofficient[1]
  b=intercept
  y=w1*x1+w2*x2+b
  return sigmoid(y)


In [ ]:
predicted(x_train_scaled.age[7],x_train_scaled.affordibility[7])

array([0.77737725], dtype=float32)

In [ ]:
def loss(y_true,y_predicted):
  epsilon=1e-12
  y_predict = [max(i,epsilon) for i in y_predicted]
  y_predict = [min(i,1-epsilon) for i in y_predict]
  y_predicted_new = np.array(y_predict)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
class gradient_descent:
  def __init__(self):
    self.w1=self.w2=1
    self.b=0
    
    
  def fit(self,x,y_train,epochs):
    self.w1,self.w2,self.b=self.predicted(x['age'],x['affordibility'],y_train,epochs)
    print(f'final values are cofficient1 = {self.w1},cofficient2 = {self.w2},bias = {self.b}')

  def predicted(self,age,affordibility,y_train,epochs):
    n=len(y_train)
    rate=0.2
    for i in range(epochs):
      y= self.w1*age + self.w2*affordibility + self.b
      y_predicted=sigmoid(y)
      losses=loss(y_train,y_predicted)
      
      w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_train)) 
      w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_train)) 
      bd = np.mean(y_predicted-y_train)

      self.w1=self.w1-rate*w1d
      self.w2=self.w2-rate*w2d
      self.b =self.b-rate*bd

      if i%50==0:
        print(i,self.w1,self.w2,self.b,losses)

    return self.w1, self.w2, self.b

  def predict(self, X_test):
    weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.b
    return sigmoid(weighted_sum)


In [ ]:
Gradient=gradient_descent()
Gradient.fit(x_train_scaled,y_train,2000)

0 0.9957123386047189 0.977729565809415 -0.04759901443866844 0.6686216072654623
50 1.3927830361278635 0.7823634941555857 -0.9946798799312482 0.5080758226856533
100 1.9110005733924125 0.8974849897087873 -1.3676831097863644 0.46553537805094736
150 2.3382282758082664 0.9994353582213914 -1.6652658320303462 0.43713959408250086
200 2.6925629101267137 1.0886402871037788 -1.9143208845138338 0.41743388695725475
250 2.990420717182138 1.168694508155768 -2.1269806549056307 0.40330192516456553
300 3.243908706699549 1.2417062237504495 -2.311378163826552 0.3928789722198861
350 3.4620227285370104 1.3089428745710825 -2.4732368489417134 0.38500591019423247
400 3.651543629601465 1.3712209110770264 -2.6167185922980525 0.37893670547420927
450 3.8176575338909617 1.4291083077415672 -2.74493794476683 0.37417594160034395
500 3.9643861710781216 1.4830310645716696 -2.86028499279162 0.3703853095778044
550 4.0948887618527605 1.5333303671838625 -2.964634389838354 0.3673279531503498
600 4.21167672407333 1.58029388537

In [ ]:
X_test_scaled = x_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [ ]:
Gradient.predict(X_test_scaled)

10    0.274454
23    0.620416
27    0.633095
24    0.681859
13    0.072639
25    0.726937
dtype: float64

In [ ]:
y_test

10    0
23    1
27    0
24    1
13    0
25    1
Name: bought_insurance, dtype: int64